#### metaclass

* yaml
    * 设计模式优点:完全不需要提前知道任何类型信息，让超动态配置编程成了可能
    * YAML 的动态序列化 / 逆序列化是由 metaclass 实现
        - 利用 YAMLObjectMetaclass 的__init__方法，为所有 YAMLObject 子类偷偷执行add_constructor()
        - YAML 应用 metaclass，拦截了所有 YAMLObject 子类的定义 `cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)`
* metaclass
    - 所有的 Python 的用户定义类，都是 type 这个类的实例
    - 用户自定义类，是 type 类的__call__运算符重载 `
class = type(classname, superclasses, attributedict)`
    - metaclass 是 type 的子类，通过替换 type 的__call__运算符重载机制，“超越变形”正常的类 `class = MyMeta(classname, superclasses, attributedict)`
 * metaclass 仅仅是给小部分 Python 开发者，在开发框架层面的 Python 库时使用的。而在应用层，metaclass 往往不是很好的选择

In [8]:
import yaml

class Monster(yaml.YAMLObject):
  yaml_tag = u'!Monster'

  def __init__(self, name, hp, ac, attacks):
    self.name = name
    self.hp = hp
    self.ac = ac
    self.attacks = attacks
    
  def __repr__(self):
    return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
       self.__class__.__name__, self.name, self.hp, self.ac,      
       self.attacks)

yaml.load("""
--- !Monster
name: Cave spider
hp: [2,6]    # 2d6
ac: 16
attacks: [BITE, HURT]
""")

print(yaml.dump(Monster(
    name='Cave lizard', hp=[3,6], ac=16, attacks=['BITE','HURT'])))

!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard



/home/henry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [12]:
# 类本身是一个名为 type 类的实例
class MyClass:
  pass

instance = MyClass()

print(type(instance))
print(type(MyClass))

<class '__main__.MyClass'>
<class 'type'>


In [21]:
class MyClass:
  data = 1
  
instance = MyClass()
MyClass, instance
print(instance.data)

# 底层
MyClass = type('MyClass', (), {'data': 1})
instance = MyClass()
MyClass, instance
print(instance.data)

1


In [22]:
class Mymeta(type):
    def __init__(self, name, bases, dic):
        super().__init__(name, bases, dic)
        print('===>Mymeta.__init__')
        print(self.__name__)
        print(dic)
        print(self.yaml_tag)

    def __new__(cls, *args, **kwargs):
        print('===>Mymeta.__new__')
        print(cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('===>Mymeta.__call__')
        obj = cls.__new__(cls)
        cls.__init__(cls, *args, **kwargs)
        return obj

class Foo(metaclass=Mymeta):
    yaml_tag = '!Foo'

    def __init__(self, name):
        print('Foo.__init__')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)

foo = Foo('foo')

===>Mymeta.__new__
Mymeta
===>Mymeta.__init__
Foo
{'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x7fa7fb130dd0>, '__new__': <function Foo.__new__ at 0x7fa7fb1304d0>}
!Foo
===>Mymeta.__call__
Foo.__new__
Foo.__init__
